# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [5]:
#DFS Solution

from itertools import combinations

LOCATIONS = ['Left', 'Island', 'Rigth']

def is_valid(state):
    H1, W1, H2, W2, _ = state

    for (W, H_own, H_other) in [(W1, H1, H2), (W2, H2, H1)]:
        if W != H_own and W == H_other:
            return False
    return True

def move(state, persons, from_loc, to_loc):
    new_state = list(state)
    for p in persons:
        new_state[p] = to_loc
    new_state[4] = to_loc  
    return tuple(new_state)

def get_possible_moves(state):
    positions = list(state)
    boat_pos = positions[4]
    people_on_bank = [i for i, pos in enumerate(positions[:4]) if pos == boat_pos]

    for r in [1, 2]:
        for group in combinations(people_on_bank, r):
            for target in ['Left', 'Island', 'Right']:
                if target != boat_pos:
                    new_state = move(positions, group, boat_pos, target)
                    if is_valid(new_state):
                        yield new_state

def is_goal(state):
    return all(pos == 'Right' for pos in state[:4])

def dfs(state, visited, path):
    if is_goal(state):
        return path + [state]

    visited.add(state)
    for next_state in get_possible_moves(state):
        if next_state not in visited:
            result = dfs(next_state, visited, path + [state])
            if result:
                return result
    return None

initial_state = ('Left', 'Left', 'Left', 'Left', 'Left') 
solution = dfs(initial_state, set(), [])

if solution:
    for step in solution:
        print(step)
else:
    print("No solution found.")


('Left', 'Left', 'Left', 'Left', 'Left')
('Island', 'Island', 'Left', 'Left', 'Island')
('Left', 'Island', 'Left', 'Left', 'Left')
('Left', 'Island', 'Left', 'Island', 'Island')
('Left', 'Left', 'Left', 'Island', 'Left')
('Left', 'Left', 'Island', 'Island', 'Island')
('Left', 'Left', 'Right', 'Right', 'Right')
('Left', 'Left', 'Left', 'Right', 'Left')
('Left', 'Island', 'Left', 'Right', 'Island')
('Left', 'Right', 'Left', 'Right', 'Right')
('Left', 'Right', 'Left', 'Left', 'Left')
('Island', 'Right', 'Island', 'Left', 'Island')
('Right', 'Right', 'Island', 'Left', 'Right')
('Right', 'Left', 'Island', 'Left', 'Left')
('Right', 'Left', 'Island', 'Island', 'Island')
('Right', 'Left', 'Right', 'Island', 'Right')
('Left', 'Left', 'Right', 'Island', 'Left')
('Left', 'Island', 'Right', 'Island', 'Island')
('Left', 'Island', 'Right', 'Right', 'Right')
('Left', 'Island', 'Left', 'Right', 'Left')
('Island', 'Island', 'Left', 'Right', 'Island')
('Island', 'Right', 'Left', 'Right', 'Right')
('Isla

In [4]:
# Backtracking solution

from copy import deepcopy

class State:
    def __init__(self, left, island, right, boat_location, path=None):
        self.left = left
        self.island = island
        self.right = right
        self.boat_location = boat_location
        self.path = path or []

    def is_goal(self, total_people):
        return len(self.right) == total_people

    def is_valid(self):
        return all(self._is_group_safe(g) for g in [self.left, self.island, self.right])

    def _is_group_safe(self, group):
        husbands = {p for p in group if p.startswith('H')}
        wives = {p for p in group if p.startswith('W')}
        for w in wives:
            w_index = w[1]
            h = 'H' + w_index
            if h not in group and any(hh != h for hh in husbands):
                return False
        return True

    def get_key(self):
        return (
            tuple(sorted(self.left)),
            tuple(sorted(self.island)),
            tuple(sorted(self.right)),
            self.boat_location
        )

    def clone(self, move, from_loc, to_loc, next_boat_loc):
        new_state = deepcopy(self)
        from_list = getattr(new_state, from_loc)
        to_list = getattr(new_state, to_loc)

        for person in move:
            from_list.remove(person)
            to_list.append(person)

        new_state.boat_location = next_boat_loc
        new_state.path.append(new_state.get_key())

        return new_state

def generate_moves(group):
    moves = []
    for i in range(len(group)):
        moves.append([group[i]])
        for j in range(i + 1, len(group)):
            moves.append([group[i], group[j]])
    return moves

def get_possible_moves(state):
    moves = []
    if state.boat_location == "left":
        group = state.left
        from_loc, to_loc = "left", "island"
        next_boat_loc = "island"
    elif state.boat_location == "island":
        for direction in [("island", "right", "right"), ("island", "left", "left")]:
            group = getattr(state, direction[0])
            for move in generate_moves(group):
                if len(move) >= 1:
                    new_state = state.clone(move, direction[0], direction[1], direction[2])
                    if new_state.is_valid():
                        moves.append(new_state)
        return moves
    elif state.boat_location == "right":
        group = state.right
        from_loc, to_loc = "right", "island"
        next_boat_loc = "island"
    else:
        return []

    for move in generate_moves(group):
        if len(move) >= 1:
            new_state = state.clone(move, from_loc, to_loc, next_boat_loc)
            if new_state.is_valid():
                moves.append(new_state)
    return moves

def backtrack(state, visited, total_people):
    if state.is_goal(total_people):
        return state.path + [state.get_key()]

    visited.add(state.get_key())
    for next_state in get_possible_moves(state):
        key = next_state.get_key()
        if key not in visited:
            result = backtrack(next_state, visited, total_people)
            if result:
                return result
    return None

def solve_jealous_husbands():
    n = 3
    people = [f'H{i+1}' for i in range(n)] + [f'W{i+1}' for i in range(n)]
    initial_state = State(left=people, island=[], right=[], boat_location="left")
    solution = backtrack(initial_state, set(), total_people=len(people))
    return solution

solution = solve_jealous_husbands()
if solution:
    for step in solution:
        print(step)
else:
    print("No solution found.")



(('H2', 'H3', 'W2', 'W3'), ('H1', 'W1'), (), 'island')
(('H2', 'H3', 'W2', 'W3'), (), ('H1', 'W1'), 'right')
(('H2', 'H3', 'W2', 'W3'), ('H1',), ('W1',), 'island')
(('H1', 'H2', 'H3', 'W2', 'W3'), (), ('W1',), 'left')
(('H1', 'H3', 'W3'), ('H2', 'W2'), ('W1',), 'island')
(('H1', 'H2', 'H3', 'W3'), ('W2',), ('W1',), 'left')
(('H1', 'H2', 'H3'), ('W2', 'W3'), ('W1',), 'island')
(('H1', 'H2', 'H3'), ('W3',), ('W1', 'W2'), 'right')
(('H1', 'H2', 'H3'), ('W1', 'W3'), ('W2',), 'island')
(('H1', 'H2', 'H3'), ('W1',), ('W2', 'W3'), 'right')
(('H1', 'H2', 'H3'), ('W1', 'W2'), ('W3',), 'island')
(('H1', 'H2', 'H3'), ('W2',), ('W1', 'W3'), 'right')
(('H1', 'H2', 'H3'), ('W1', 'W2', 'W3'), (), 'island')
(('H1', 'H2', 'H3', 'W2'), ('W1', 'W3'), (), 'left')
(('H2', 'W2'), ('H1', 'H3', 'W1', 'W3'), (), 'island')
(('H2', 'W2'), ('H1', 'H3'), ('W1', 'W3'), 'right')
(('H2', 'W2'), ('H1', 'H3', 'W3'), ('W1',), 'island')
(('H2', 'W2'), ('W3',), ('H1', 'H3', 'W1'), 'right')
(('H2', 'W2'), ('W1', 'W3'), ('H

In [1]:
# A* solution
#heuristic - number -of ppl not on the right - h(x)
#f(x) = g(x) + h(x) , g(x) - total number of moves made so far
import heapq
from copy import deepcopy

class State:
    def __init__(self, left, island, right, boat_location, path=None, g=0):
        self.left = left
        self.island = island
        self.right = right
        self.boat_location = boat_location
        self.path = path or []
        self.g = g  

    def is_goal(self, total_people):
        return len(self.right) == total_people

    def is_valid(self):
        return all(self._is_group_safe(g) for g in [self.left, self.island, self.right])

    def _is_group_safe(self, group):
        husbands = {p for p in group if p.startswith('H')}
        wives = {p for p in group if p.startswith('W')}
        for w in wives:
            w_index = w[1]
            h = 'H' + w_index
            if h not in group and any(hh != h for hh in husbands):
                return False
        return True

    def get_key(self):
        return (
            tuple(sorted(self.left)),
            tuple(sorted(self.island)),
            tuple(sorted(self.right)),
            self.boat_location
        )

    def clone(self, move, from_loc, to_loc, next_boat_loc):
        new_state = deepcopy(self)
        from_list = getattr(new_state, from_loc)
        to_list = getattr(new_state, to_loc)

        for person in move:
            from_list.remove(person)
            to_list.append(person)

        new_state.boat_location = next_boat_loc
        new_state.path.append(self.get_key())
        new_state.g += 1
        return new_state

    def heuristic(self, total_people):
        return total_people - len(self.right)

    def f_score(self, total_people):
        return self.g + self.heuristic(total_people)

    def __lt__(self, other):
        return self.f_score(6) < other.f_score(6)

def generate_moves(group):
    moves = []
    for i in range(len(group)):
        moves.append([group[i]])
        for j in range(i + 1, len(group)):
            moves.append([group[i], group[j]])
    return moves

def get_possible_moves(state):
    moves = []
    if state.boat_location == "left":
        group = state.left
        from_loc, to_loc = "left", "island"
        next_boat_loc = "island"
    elif state.boat_location == "island":
        for direction in [("island", "right", "right"), ("island", "left", "left")]:
            group = getattr(state, direction[0])
            for move in generate_moves(group):
                if len(move) >= 1:
                    new_state = state.clone(move, direction[0], direction[1], direction[2])
                    if new_state.is_valid():
                        moves.append(new_state)
        return moves
    elif state.boat_location == "right":
        group = state.right
        from_loc, to_loc = "right", "island"
        next_boat_loc = "island"
    else:
        return []

    for move in generate_moves(group):
        if len(move) >= 1:
            new_state = state.clone(move, from_loc, to_loc, next_boat_loc)
            if new_state.is_valid():
                moves.append(new_state)
    return moves

def solve_with_astar():
    n = 3
    total_people = 2 * n
    people = [f'H{i+1}' for i in range(n)] + [f'W{i+1}' for i in range(n)]
    initial_state = State(left=people, island=[], right=[], boat_location="left")

    frontier = []
    heapq.heappush(frontier, (initial_state.f_score(total_people), initial_state))
    visited = set()

    while frontier:
        _, current = heapq.heappop(frontier)
        key = current.get_key()

        if key in visited:
            continue
        visited.add(key)

        if current.is_goal(total_people):
            return current.path + [current.get_key()]

        for neighbor in get_possible_moves(current):
            if neighbor.get_key() not in visited:
                heapq.heappush(frontier, (neighbor.f_score(total_people), neighbor))

    return None
    
solution = solve_with_astar()
if solution:
    for step in solution:
        print(step)
else:
    print("No solution found.")


(('H1', 'H2', 'H3', 'W1', 'W2', 'W3'), (), (), 'left')
(('H1', 'H2', 'H3', 'W2'), ('W1', 'W3'), (), 'island')
(('H1', 'H2', 'H3', 'W2', 'W3'), ('W1',), (), 'left')
(('H1', 'H2', 'H3'), ('W1', 'W2', 'W3'), (), 'island')
(('H1', 'H2', 'H3', 'W3'), ('W1', 'W2'), (), 'left')
(('H3', 'W3'), ('H1', 'H2', 'W1', 'W2'), (), 'island')
(('H3', 'W3'), ('H1', 'W1'), ('H2', 'W2'), 'right')
(('H3', 'W3'), ('H1', 'H2', 'W1'), ('W2',), 'island')
(('H2', 'H3', 'W3'), ('H1', 'W1'), ('W2',), 'left')
(('W3',), ('H1', 'H2', 'H3', 'W1'), ('W2',), 'island')
(('W3',), ('H1', 'W1'), ('H2', 'H3', 'W2'), 'right')
(('W3',), ('H1', 'H3', 'W1'), ('H2', 'W2'), 'island')
(('W3',), ('W1',), ('H1', 'H2', 'H3', 'W2'), 'right')
(('W3',), ('W1', 'W2'), ('H1', 'H2', 'H3'), 'island')
(('W2', 'W3'), ('W1',), ('H1', 'H2', 'H3'), 'left')
((), ('W1', 'W2', 'W3'), ('H1', 'H2', 'H3'), 'island')
((), ('W2',), ('H1', 'H2', 'H3', 'W1', 'W3'), 'right')
((), ('W1', 'W2'), ('H1', 'H2', 'H3', 'W3'), 'island')
((), (), ('H1', 'H2', 'H3', 